Анализ данных не примере первого варианта финального проекта курса "Аналитик данных" karpov.courses

[0. Retention – один из самых важных показателей в компании. Ваша задача – написать функцию, которая будет считать retention игроков (по дням от даты регистрации игрока).](#0)

[1. Имеются результаты A/B теста, в котором двум группам пользователей предлагались различные наборы акционных предложений. Известно, что ARPU в тестовой группе выше на 5%, чем в контрольной. При этом в контрольной группе 1928 игроков из 202103 оказались платящими, а в тестовой – 1805 из 202667. Какой набор предложений можно считать лучшим? Какие метрики стоит проанализировать для принятия правильного решения и как?](#1)

[2. В игре Plants & Gardens каждый месяц проводятся тематические события, ограниченные по времени. В них игроки могут получить уникальные предметы для сада и персонажей, дополнительные монеты или бонусы. Для получения награды требуется пройти ряд уровней за определенное время. С помощью каких метрик можно оценить результаты последнего прошедшего события?

Предположим, в другом событии мы усложнили механику событий так, что при каждой неудачной попытке выполнения уровня игрок будет откатываться на несколько уровней назад. Изменится ли набор метрик оценки результата? Если да, то как?](#2)

<a id="1"></a>


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
sns.set(font_scale=2, style="whitegrid", rc={'figure.figsize' : (15,7)})
import matplotlib.pyplot as plt
from scipy import special
plt.style.use('dark_background')

<a id="0"></a> Retention – один из самых важных показателей в компании. Ваша задача – написать функцию, которая будет считать retention игроков (по дням от даты регистрации игрока)

In [36]:
url='https://drive.google.com/file/d/18Fa0cDVGq7bf8O7KEb23CFgU0w5qsMaD/view?usp=sharing'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]
df_r = pd.read_csv(url, sep=";")
df_r.head(1)

,reg_ts,uid
0,911382223,1


In [37]:
url='https://drive.usercontent.google.com/download?id=12xLu8LW-RvixxIPkourv73x4IUbqpYow&export=download&authuser=0&confirm=t&uuid=d43e5d1c-cec3-4518-84ae-d49b30d9c527&at=AENtkXYRTcBqWPAkd3eLSmLq1DSN%3A1733307676786'
df_a = pd.read_csv(url, sep=";")
df_a.head(1)

,auth_ts,uid
0,911382223,1


Подготовка и знакомство с данными

In [39]:
#Таймстэмпы оказались импортированы как целые числа, переведем их даты и переименуем колонки в привычные, простые и запоминающиеся

df_r.reg_ts = pd.to_datetime(df_r.reg_ts, unit='s')
df_r = df_r.rename(columns = {'reg_ts' : 'r_date',
                     'uid' : 'id'})
df_a.auth_ts = pd.to_datetime(df_a.auth_ts, unit='s')
df_a = df_a.rename(columns = {'auth_ts' : 'a_date',
                     'uid' : 'id'})

In [110]:
# Соберем два набора данных в один по айди пользователя - допишем каждому id дату и время его регистрации
df = pd.merge(df_a, df_r, how = 'left', on='id')

В оформлении исследования данных есть такой момент - если оставлять всю последовательность знакомства с данными, то получается 10 экранов типовых проверок. КОтроые надо прокручивать потом туда-сюда и которые сами по сбее никому неинтересны, ибо банальны и очевидны.

Мне пришло в голову собрать это все в один-два бюлока через f-строку и как-будто стало компактнее, при этом с сохранением все расчеты проверяемы.

Как это вообще с точки зрения практики - применимо? 

In [117]:
df_tmp=df.groupby('id').a_date.count().sort_values(ascending = False)
print(f'\
1. Временной интервал набора данных: \n\
                                        начало:  {df['r_date'].dt.date.min()} \n\
                                        конец:   {df['r_date'].dt.date.max()} \n\
2. Уникальных пользователей:                     {df_r.id.nunique()} \n\
3. Зарегистрировались, но в игру так и не зашли: {pd.merge(df_r, df_a, how='left', on='id').a_date.isna().sum()}\n\
4. Зашли в игру один раз                         {df_tmp[df_tmp.values == 1].count()} пользователей  ({round(100*df_tmp[df_tmp.values == 1].count()/len(df_r), 1)}%) \n\
5. Зашли в игру больше одного раза:              {df_tmp[df_tmp.values > 1].count()} пользователей  ({round(100*df_tmp[df_tmp.values > 1].count()/len(df_r), 1)}%) \n\
6. NaN в наборе данных                           {df.isna().a_date.sum()+df.isna().r_date.sum()+df.isna().id.sum()} \n\
')


1. Временной интервал набора данных: 
                                        начало:  1998-11-18 
                                        конец:   2020-09-23 
2. Уникальных пользователей:                     1000000 
3. Зарегистрировались, но в игру так и не зашли: 0
4. Зашли в игру один раз                         761622 пользователей  (76.2%) 
5. Зашли в игру больше одного раза:              238378 пользователей  (23.8%) 
6. NaN в наборе данных                           0 

